In [1]:
# Imports
import matplotlib.pyplot as plt
import numpy as np
import nibabel as nib
from time import perf_counter as time
from scipy.interpolate import RegularGridInterpolator
from matplotlib.colors import LightSource
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
from skimage import measure
import scipy.ndimage
import pickle
import trimesh
from pygel3d import hmesh, jupyter_display as jd
import plotly.graph_objs as go
from commons.utils import *
from commons.display import *

In [2]:
%load_ext autoreload
%autoreload 2

## Load volumetric data

In [3]:
volume_data_path = 'data/vindelev/X19_resampled_uint8.nii'
niiVol = nib.load(volume_data_path)

In [17]:
def apply_max_in_region(arr, x_thresh, y_thresh, z_thresh=np.inf):
    """Takes a 3d binary matrix arr and merges cells in small neighborhood by taking the OR value"""
    result = np.copy(arr)

    for x in range(arr.shape[0]):
        for y in range(arr.shape[1]):
            for z in range(arr.shape[2]):
                if x < x_thresh and y < y_thresh and z < z_thresh:
                    # Apply the logical OR within a smaller neighborhood
                    x_min, x_max = max(0, x - 2), min(arr.shape[0], x + 2)
                    y_min, y_max = max(0, y - 2), min(arr.shape[1], y + 2)
                    z_min, z_max = max(0, z - 2), min(arr.shape[2], z + 2)
                    result[x, y, z] = np.max(arr[x_min:x_max, y_min:y_max, z_min:z_max])
                # For values outside threshold keep the original value

    padded_arr = np.pad(result, pad_width=2, mode='constant', constant_values=False)

    return padded_arr

In [5]:
def rescale_max(data, scale_factor, cval=0.0):
    pad_width = [(0, (scale_factor - dim % scale_factor) % scale_factor) for dim in data.shape]
    
    padded_data = np.pad(data, pad_width, mode='constant', constant_values=cval)
    
    new_shape = (padded_data.shape[0] // scale_factor, scale_factor,
                 padded_data.shape[1] // scale_factor, scale_factor,
                 padded_data.shape[2] // scale_factor, scale_factor)
    
    reshaped_data = padded_data.reshape(new_shape)
    reshaped_data = reshaped_data.max(axis=1).max(axis=2).max(axis=3)
    
    return reshaped_data

In [18]:
affine = niiVol.affine
imgSpacing = niiVol.header['pixdim'][1:4]

# Downsample
downsample_factor = 12
affine[0, 0] = affine[0, 0] * downsample_factor
affine[1, 1] = affine[1, 1] * downsample_factor
affine[2, 2] = affine[2, 2] * downsample_factor
imgSpacing = imgSpacing * downsample_factor

vol = niiVol.get_fdata().astype('float32') #scipy.ndimage.zoom(niiVol.get_fdata().astype('float32'), 1 / downsample_factor, order=2)
vol = rescale_max(vol, downsample_factor)
imgDim = vol.shape

In [19]:
res = apply_max_in_region(vol, 100, 120, 60)

In [20]:
threshold = 130
vertices_f, faces_f, _,_ = measure.marching_cubes(res > threshold , 0.5)
trim = trimesh.Trimesh(vertices_f, faces_f)

connected_components = list(trim.split(only_watertight=False))
connected_components.sort(key=lambda x: len(x.faces), reverse=True)
largest_component = connected_components[0]

In [21]:
m = trimesh_to_manifold(largest_component)
# jd.display(m)

In [22]:
smoothed = hmesh.Manifold(m)
smooth(smoothed, max_iter=40)
# hmesh.quadric_simplify(smoothed, 0.7)
# smoothed.cleanup()
jd.display(smoothed)

FigureWidget({
    'data': [{'color': '#dddddd',
              'flatshading': False,
              'i': array([    0,  1979,     1, ..., 85695, 84830, 86125]),
              'j': array([58160,     1,   100, ..., 85704, 85701, 86136]),
              'k': array([58216,     2,     2, ..., 86125, 86067, 85695]),
              'type': 'mesh3d',
              'uid': 'd377d691-6717-4995-9255-cc54eb89fc21',
              'x': array([ 8.243047  , 98.90749675, 99.0224553 , ..., 60.81875765, 61.72792847,
                          61.70226988]),
              'y': array([ 46.56330445,  31.91128641,  31.87554667, ..., 100.32576542,
                           85.15395816, 100.31327079]),
              'z': array([70.81687936, 53.02628619, 52.55768971, ..., 31.27335124, 66.71382816,
                          28.09269053])},
             {'hoverinfo': 'none',
              'line': {'color': 'rgb(125,0,0)', 'width': 1},
              'mode': 'lines',
              'type': 'scatter3d',
              'ui

In [46]:
hmesh.quadric_simplify(smoothed, 0.5)
hmesh.triangulate(smoothed)
hmesh.flip_orientation(smoothed)
smoothed.cleanup()

In [47]:
jd.display(smoothed)

FigureWidget({
    'data': [{'color': '#dddddd',
              'flatshading': False,
              'i': array([    0,     1,    12, ..., 41026, 42933, 42968]),
              'j': array([30627,  3025,     1, ..., 42249, 42932, 42249]),
              'k': array([30582,     2,     2, ..., 42270, 42163, 41026]),
              'type': 'mesh3d',
              'uid': '7e511d8e-1ffe-4756-9254-e970f2a70e54',
              'x': array([ 8.24689899, 97.07290547, 97.56529548, ..., 62.18336203, 63.17472736,
                          62.980594  ]),
              'y': array([ 46.56657802,  23.96299103,  25.20802397, ..., 118.14443373,
                           69.0067552 , 119.30355124]),
              'z': array([70.81536323, 50.7526936 , 52.25511055, ..., 59.11804666, 56.73657204,
                          58.62126573])},
             {'hoverinfo': 'none',
              'line': {'color': 'rgb(125,0,0)', 'width': 1},
              'mode': 'lines',
              'type': 'scatter3d',
              'ui

In [48]:
hmesh.obj_save("x19_zoom_mesh.obj", smoothed)